In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from glob import glob

In [5]:
startech_df = pd.read_csv("./datasets/startech.csv")
# daraz_df = pd.read_csv("./datasets/daraz.csv")

# print(daraz_df.columns, startech_df.columns)
startech_df.head(5)

,product_name,price,product_brand,product_key_features,product_specifications,product_description
0,Apple MacBook Pro 16 inch M4 Pro Chip Liquid R...,379500,Apple,"MPN: MX2Y3LL/A,Model: MacBook Pro M4 Pro,Proce...",Processor Brand: Apple\nProcessor Model: M4 Pr...,Apple MacBook Pro 16 inch M4 Pro Chip Liquid R...
1,Lenovo Legion Slim 5 16AHP9 Ryzen 7 8845HS AI ...,200000,Lenovo,"MPN: 83DH003DLK,Model: Legion Slim 5 16AHP9,Pr...",Processor Brand: AMD\nProcessor Model: Ryzen 7...,Lenovo Legion Slim 5 16AHP9 Ryzen 7 8845HS AI ...
2,Lenovo LOQ 15AHP9 Ryzen 7 8845HS AI Integrated...,175000,Lenovo,"MPN: 83DX00A6LK,Model: LOQ 15AHP9,Processor: A...",Processor Brand: AMD\nProcessor Model: Ryzen 7...,Lenovo LOQ 15AHP9 Ryzen 7 8845HS RTX 4060 8GB ...
3,Lenovo LOQ 15AHP9 Ryzen 7 8845HS AI Integrated...,165000,Lenovo,"MPN: 83DX00A4LK,Model: LOQ 15AHP9,Processor: A...",Processor Brand: AMD\nProcessor Model: Ryzen 7...,Lenovo LOQ 15AHP9 Ryzen 7 8845HS RTX 4050 6GB ...
4,Lenovo IdeaPad Pro 5 14AHP9 Ryzen 7 8845HS AI ...,175000,Lenovo,"MPN: 83D30031LK,Model: IdeaPad Pro 5 14AHP9,Pr...",Processor Brand: AMD\nProcessor Model: Ryzen 7...,Lenovo IdeaPad Pro 5 14AHP9 Ryzen 7 8845HS RTX...


In [ ]:
startech_df = startech_df.dropna(subset=["price"]).reset_index(drop=True)
startech_df = startech_df[startech_df["price"] != "To be announce"].copy()
startech_df = startech_df
startech_df = startech_df[["product_name", "price", "product_specifications"]].copy()

In [ ]:
startech_df["price"] = startech_df["price"].replace(",", "", regex=True)
startech_df = startech_df.rename(
    columns={"product_name": "title", "product_specifications": "description"}
)
print(startech_df.shape)
startech_df.head(5)

In [ ]:
final_5000_products = startech_df.sample(n=5000, random_state=42).reset_index(drop=True)
final_5000_products.to_csv("./final_5000_products.csv", index=False)

In [ ]:
bengali_alphabets = "অআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহয়ড়ঢ়ংঃঁ"
bengali_pattern = re.compile(f"[{bengali_alphabets}]")

bengali_daraz_df = daraz_df[
    daraz_df["title"].apply(lambda x: bool(bengali_pattern.search(x)))
    & daraz_df["description"].apply(lambda x: bool(bengali_pattern.search(x)))
].reset_index(drop=True)

In [ ]:
len_bengali_df = bengali_daraz_df.shape[0]
need = 5000 - len_bengali_df

In [ ]:
sampled_startech_df = startech_df.sample(
    n=need, random_state=42, replace=False
).reset_index(drop=True)

In [ ]:
final_product_df = pd.concat([sampled_startech_df, bengali_daraz_df], axis=0)
print(final_product_df.shape)
final_product_df.head(5)

In [ ]:
final_product_df.to_csv("./final_5000_products.csv", index=False)

## Join generated queries dataframe with main dataset


In [5]:
def format_product_details(name, price, description):
    product_details = ""
    if description is not None:
        product_details = f"Name: {name}\nPrice: {price} taka\n{description}"
    else:
        product_details = f"Name: {name}\nPrice: {price} taka"

    return product_details

In [2]:
query_df = [pd.read_csv(path)for path in glob("./datasets/generated_data/*.csv")]
query_df = pd.concat(query_df)
print(query_df.shape)
query_df.head(5)

(24761, 3)


,product_id,relevant_query,irrelevant_query
0,009b7e66-ef69-49fc-87c8-9d40d53e0e33,Basic calculator,Best gift for a teacher
1,009b7e66-ef69-49fc-87c8-9d40d53e0e33,check and recheck function,Calculator reviews 2023
2,009b7e66-ef69-49fc-87c8-9d40d53e0e33,digital display calculator,How to use a calculator
3,009b7e66-ef69-49fc-87c8-9d40d53e0e33,handheld calculator,Calculator prices in Bangladesh
4,009b7e66-ef69-49fc-87c8-9d40d53e0e33,simple math calculator,Digital watch


In [3]:
product_df = pd.read_csv("./datasets/final_5000_products.csv")
product_df.head(5)

,title,price,description,id
0,Casio DJ-120D Plus Check & Recheck Basic Calcu...,1305,NaN,009b7e66-ef69-49fc-87c8-9d40d53e0e33
1,Colorful CN600 PRO 1TB M.2 NVMe SSD,7300,Capacity: 1TB\nFlash Type: 3D NAND\nInterface:...,7bd5da56-89e9-4b68-92e2-cd31f0578bcb
2,Anker Soundcore Space One Foldable Over-Ear Bl...,7990,Frequency Range: 20Hz-20KHz\nInput Jack: AUX C...,3c7d8f65-a7b7-47cd-b808-d6e8c445ca69
3,"Smart SEL-50V24K 50"" 4K Voice Control Android ...",51900,Display Type: LED\nScreen Size: 50 Inch\nResol...,212bc014-cec5-4bc6-ad82-2591098ab808
4,EZVIZ H3c 3MP Wi-Fi Smart Home Outdoor Securit...,4324,Image Sensor: 1/2.7”Progressive Scan CMOS\nEff...,617e0e00-cfd2-4465-b46f-9537476327a4


In [6]:
merged_df = (
    pd.merge(product_df, query_df, left_on="id", right_on="product_id")
    .drop(columns=["id", "product_id"])
    .reset_index(drop=True)
)
merged_df["anchor"] = merged_df.apply(
    lambda row: format_product_details(
        name=row["title"], description=row["description"], price=row["price"]
    ),
    axis=1,
)
merged_df = merged_df.drop(columns=["title", "price", "description"]).rename(columns={"relevant_query": "positive", "irrelevant_query": "negative"})
merged_df = merged_df[["anchor", "positive", "negative"]]
print(merged_df.shape)
merged_df.tail(5)

(24761, 3)


,anchor,positive,negative
24756,Name: Redragon Storm Pro M808-KS RGB USB 2.4G ...,wireless gaming mouse,best gaming keyboard for Fortnite
24757,Name: Redragon Storm Pro M808-KS RGB USB 2.4G ...,lightweight gaming mouse,wireless earbuds for music
24758,Name: Redragon Storm Pro M808-KS RGB USB 2.4G ...,high polling rate gaming mouse,portable power bank for phone
24759,Name: Redragon Storm Pro M808-KS RGB USB 2.4G ...,programmable buttons gaming mouse,black gaming chair for sale
24760,Name: Redragon Storm Pro M808-KS RGB USB 2.4G ...,gaming mouse with high DPI indicator,gaming headset with microphone


## Dataset split


In [7]:
train_df, val_df = train_test_split(
    merged_df, test_size=0.10, random_state=42, shuffle=True
)
train_df, test_df = train_test_split(
    train_df, test_size=0.20, random_state=42, shuffle=True
)

train_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

train_df.shape, val_df.shape, test_df.shape

((17827, 3), (2477, 3), (4457, 3))

In [8]:
train_df.to_csv("./datasets/train.csv", index=False)
test_df.to_csv("./datasets/test.csv", index=False)
val_df.to_csv("./datasets/val.csv", index=False)